In [ ]:
import cobra
import requests
from Bio import SeqIO
import pandas as pd
import re
import time

# fetch_pr

In [ ]:

def fetch_pr(taxon_id, reviewed=False, format='fasta', limit=1000):
    base_url = "https://rest.uniprot.org/uniprotkb/stream"
    query = f"organism_id:{taxon_id}"
    if reviewed:
        query += " AND reviewed:true"

    params = {"query": query, "format": format, "size": limit}

    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        with open(f"/home/aac/Mohammad/GNN/MetaBiomeX-main/data/fastas/uniprot_{taxon_id}.fasta", "w") as f:
            f.write(response.text)
        return response.text 
    else:
        return None

In [ ]:
models_list= pd.read_csv('/home/aac/Mohammad/GNN/MetaBiomeX-main/data/ListofModelsforSpecies.csv')

In [ ]:
xml_files = []

for entry in models_list['models'].dropna():
    found = re.findall(r'\b[\w\-\.]+\.xml\b', str(entry))
    xml_files.extend(found)

xml_files = sorted(set(xml_files))
xml_files[:5]

In [ ]:
# df['Strain_cleaned'] = df['Strain'].str.replace('.xml', '', regex=False)


base_url = "https://www.vmh.life/_api/microbes/?reconstruction="

def get_ncbiid(strain):
    response = requests.get(base_url + strain)
    if response.status_code == 200:
        data = response.json()
        if data.get("results"):
            return data["results"][0].get("ncbiid")

In [ ]:
xml_df = pd.DataFrame(xml_files, columns=['XML File'])
xml_df['strain_name'] = xml_df['XML File'].str.replace('.xml', '', regex=False)


In [ ]:
xml_df['strain_name'][0]

In [ ]:
a= get_ncbiid(xml_df['strain_name'][0])
print(a)

In [ ]:
xml_df.head()

In [ ]:
xml_df['ncbiid'] = xml_df['strain_name'].apply(get_ncbiid)

In [ ]:
xml_df.head()

In [ ]:
path = "/home/aac/Mohammad/GNN/MetaBiomeX-main/data/strain_with_NCBIids.csv"
xml_df.to_csv(path, index=False)

# Fetch Pr. seqs

In [ ]:
import pandas as pd
xml_df= pd.read_csv("/home/aac/Mohammad/GNN/MetaBiomeX-main/data/strain_with_NCBIids.csv")

In [ ]:
ncbi_ids = xml_df['ncbiid']#.dropna().unique()

In [ ]:
fetched_files = {}
from tqdm import tqdm
for taxon_id in tqdm(ncbi_ids[130:140]):
    # print(f"fetching: {taxon_id}")
    try:
        file_path = fetch_pr(taxon_id=int(taxon_id), reviewed=False)
        if file_path:
            fetched_files[taxon_id] = file_path
            # print('done')
    except Exception as e:
        fetched_files[taxon_id] = f"Error: {str(e)}"
        print('failed')



In [ ]:
ncbi_ids[130:140]

## Number of Downloaded sequences

In [ ]:
import os
import glob
from Bio import SeqIO
from tqdm import tqdm
import multiprocessing as mp

empty_fasta = []
def process_fasta(file_path):
    """
    Process a single FASTA file:
    - Count sequences
    - Determine the min and max sequence lengths in the file
    """
    total = 0
    min_length = float('inf')
    max_length = 0
    
    # Parse each record in the FASTA file
    for record in SeqIO.parse(file_path, "fasta"):
        seq_len = len(record.seq)
        total += 1
        if seq_len < min_length:
            min_length = seq_len
        if seq_len > max_length:
            max_length = seq_len
            
    # Handle files that might be empty
    if total == 0:
        min_length = 0
        
    return total, min_length, max_length,file_path

if __name__ == "__main__":
    # Specify the directory containing the FASTA files
    directory = "/home/aac/Mohammad/GNN/MetaBiomeX-main/data/fastas"  # change this to your directory path
    # Gather all FASTA files (adjust the pattern if your file extension differs)
    fasta_files = glob.glob(os.path.join(directory, "*.fasta"))
    
    total_sequences = 0
    overall_min = float('inf')
    overall_max = 0

    # Use a multiprocessing Pool to process the FASTA files concurrently
    with mp.Pool() as pool:
        # Use tqdm to track progress of the multiprocessing jobs
        results = list(tqdm(pool.imap_unordered(process_fasta, fasta_files),
                            total=len(fasta_files),
                            desc="Processing FASTA files"))
    
    # Combine the results from each file
    for total, min_length, max_length,path in results:
        if total ==0:
            empty_fasta.append(path)
        total_sequences += total
        overall_min = min(overall_min, min_length)
        overall_max = max(overall_max, max_length)
    
    # If no sequences were found overall, adjust the minimum length
    if overall_min == float('inf'):
        overall_min = 0

    print("Total sequences:", total_sequences)
    print("Min sequence length:", overall_min)
    print("Max sequence length:", overall_max)


In [ ]:
empty_ids = ([int(w.replace('.fasta','').split('_')[-1]) for w in empty_fasta])
len(empty_ids)

# Redownload empty ones

In [ ]:
fetched_files = {}
from tqdm import tqdm
for taxon_id in tqdm(empty_ids):
    # print(f"fetching: {taxon_id}")
    try:
        file_path = fetch_pr(taxon_id=int(taxon_id), reviewed=False)
        if file_path:
            fetched_files[taxon_id] = file_path
            # print('done')
    except Exception as e:
        fetched_files[taxon_id] = f"Error: {str(e)}"
        print('failed')

In [ ]:
empty_ids

# Start from here ::::::::::::

In [ ]:
node_map = pd.read_csv('anaerobic_strains_map.csv')

In [ ]:
# node_map.head()

In [ ]:
node_map[node_map['ID'] == 812]


In [ ]:
nodes = [int(x) for x in ['812', '383', '44', '334', '422', '444', '477', '534', '399', '302',
                  '141', '129', '752', '21', '268', '484', '23', '224', '78', '641',
                  '548', '549', '34', '528', '143', '118', '152', '343', '123', '72',
                  '239', '688', '150', '467', '164', '562', '13', '276', '203', '104',
                  '645', '530', '131', '24', '664', '529', '431', '209', '350', '502',
                  '540', '315', '19', '271', '119', '12', '331', '304', '349', '456',
                  '751', '200', '67', '792', '395', '656', '55', '174', '788', '192',
                  '782', '489', '326', '538', '503']]

final_nodes = node_map[node_map['ID'].isin(nodes)]

    


In [ ]:
final_nodes.shape # should be (75 in 7)

In [ ]:
import numpy as np
import time

In [ ]:
empty_ids=[469617,
 273526,
 440497,
 702444,
 1235815,
 435837,
 888048,
 457424,
 563194,
 575595,
 1121102,
 1215915,
 1400136,
 742820,
 1328388,
 411901,
 1262650,
 525337,
 1328337,
 469599,
 888727,
 702443,
 1322347,
 411487,
 469602,
 536233,
 649757,
 1353979,
 663952,
 1121342,
 469618,
 657324,
 1035196,
 1122982,
 525279,
 871541,
 1440052,
 1316931,
 469616,
 469603,
 883166,
 657316,
 1121445,
 944562,
 1203540,
 562982,
 1437595,
 1074451,
 469615,
 1384484]

In [ ]:
empty_ids_inNodes = node_map[node_map['ncbiid'].isin(np.array(empty_ids))]

In [ ]:
empty_ids_inNodes.shape # should be (50 in 7)

In [ ]:
empty_ids_inNodes.head()

In [ ]:
from Bio import Entrez
import time

Entrez.email = "javadamn@gmail.com" 

taxonomy_ids = empty_ids_inNodes['ncbiid'] #['1074451', '649757']

for tax_id in taxonomy_ids:
    print(f"\nSearching NCBI protein database for Taxonomy ID: {tax_id}")
    
    try:
        handle = Entrez.esearch(db="protein", term=f"txid{tax_id}[Organism:exp]", retmax=5000)
        record = Entrez.read(handle)
        handle.close()

        ids = record["IdList"]

        if not ids:
            print(f"No protein records {tax_id}")
            continue

        print(f"found {len(ids)} pr records")

        batch_size = 500 
        filename = f"ncbi_taxonomy_{tax_id}.fasta"
        with open(filename, "w") as out_f:
            for start in range(0, len(ids), batch_size):
                end = min(start + batch_size, len(ids))
                id_batch = ids[start:end]
                fetch_handle = Entrez.efetch(db="protein", id=id_batch, rettype="fasta", retmode="text")
                data = fetch_handle.read()
                fetch_handle.close()
                out_f.write(data)
                time.sleep(0.5)

        print(f"Saved all seqs: {filename}")

    except Exception as e:
        print(f"Error processing tax id {tax_id}: {e}")


# Fetch Genome seqs  

In [ ]:
from Bio import Entrez, SeqIO

In [ ]:
Entrez.email = "javadamn@gmai.com"

In [ ]:
data= pd.read_csv("/home/javad/pyprojects/MO_GEMs_Score/graphNN/data/strain_with_NCBIids.csv")
# xml_df= pd.read_csv('/home/aac/Mohammad/GNN/MetaBiomeX-main/data/strain_with_NCBIids.csv')

In [ ]:
data.head()

In [ ]:
def get_genome_accession(tax_id):
    handle = Entrez.esearch(db="assembly", term=f"txid{tax_id}[Organism:exp] AND latest[filter]", retmax=1)
    record = Entrez.read(handle)
    handle.close()
    if record['IdList']:
        assembly_id = record['IdList'][0]
        # Get assembly summary to find RefSeq accession
        summary_handle = Entrez.esummary(db="assembly", id=assembly_id)
        summary_record = Entrez.read(summary_handle)
        summary_handle.close()
        asm_summary = summary_record['DocumentSummarySet']['DocumentSummary'][0]
        refseq_acc = asm_summary.get("AssemblyAccession")
        return refseq_acc
    else:
        return None

In [ ]:
gen_Acc = data['ncbiid'].apply(get_genome_accession)
data['genome_accession'] = gen_Acc
# ncbi_ids = ['NC_' + str(x) for x in xml_df['ncbiid']]

# ncbi_ids =ncbi_ids[1:3]
# ncbi_ids

In [ ]:
data.head()

In [ ]:
for idx, row in data.iterrows():
    tax_id = row["ncbiid"]
    accession = row["genome_accession"]

    if not accession:
        print(f"Skipping TaxID {tax_id} due to missing accession")
        continue

    try:
        handle = Entrez.esearch(db="nuccore", term=f"{accession}[Assembly Accession] AND refseq[filter]", retmax=1)
        record = Entrez.read(handle)
        handle.close()

        if record['IdList']:
            seq_id = record['IdList'][0]
            seq_handle = Entrez.efetch(db="nuccore", id=seq_id, rettype="fasta", retmode="text")
            records = list(SeqIO.parse(seq_handle, "fasta"))
            seq_handle.close()

            if records:
                seq_record = records[0]
                filename = f"{tax_id}.fasta"
                with open(filename, "w") as output:
                    SeqIO.write(seq_record, output, "fasta")
                print(f"Saved: {filename}")
            else:
                print(f"No FASTA records found for {accession} (TaxID {tax_id})")
        else:
            print(f"No sequence ID found for {accession} (TaxID {tax_id})")

    except Exception as e:
        print(f"Error processing {accession} (TaxID {tax_id}): {e}")

    time.sleep(0.4) 

In [ ]:
for ncbi_id in ncbi_ids:
    handle = Entrez.efetch(db="nuccore", id=ncbi_id, rettype="fasta", retmode="text")
    seq_record = SeqIO.read(handle, "fasta")
    handle.close()

    # Save sequences to individual fasta files (optional)
    with open(f"{ncbi_id}.fasta", "w") as output_handle:
        SeqIO.write(seq_record, output_handle, "fasta")

    print(f"Sequence retrieved for {ncbi_id}")